# Code to Convert Recipes to JSON
#### Last Edited: Aug 8, 2025
#### run this before the app_py developer

#### Notes
- Bug: not every recipe has a temp, so in the app ingredients shift upward if temp is missing and the first ingredient fills the temp position. Likely need to adjust parser to fill missing temps with " " or n/a 

In [19]:
import json
import re

def parse_recipes_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    recipes = []
    recipe = {}
    section = None
    buffer = []
    field_idx = 0

    def clean_line(line):
        line = line.strip()
        line = line.replace('\u00a0', ' ')  # replace non-breaking space
        line = re.sub(r'^[\*\-\•\u2022\s]+', '- ', line)  # normalize bullets
        return line

    def finalize_section():
        nonlocal buffer, section
        if not buffer or not section:
            return

        cleaned = [line.lstrip("-•* ").strip() for line in buffer if line.strip()]
        if section == "ingredients":
            recipe["ingredients"] = cleaned
        elif section == "instructions":  # unified name
            recipe["instructions"] = cleaned
        elif section == "notes":
            recipe["notes"] = "\n".join(cleaned)
        elif section == "tags":
            # Split by commas and strip whitespace
            tags_list = []
            for line in cleaned:
                tags_list.extend([t.strip() for t in line.split(",") if t.strip()])
            recipe["tags"] = tags_list
        buffer = []

    for line in lines:
        line = clean_line(line)

        # Divider between recipes
        if line.startswith("_____"):
            finalize_section()
            if recipe:
                # Ensure all keys exist
                if "tags" not in recipe:
                    recipe["tags"] = []
                recipes.append(recipe)
            recipe = {}
            section = None
            field_idx = 0
            continue

        # Start of a new recipe (title line)
        if not recipe and line:
            recipe["title"] = line.strip()
            field_idx = 0
            continue

        # First three metadata lines (bulleted)
        if recipe and field_idx < 3 and line.startswith("- "):
            text = line.replace("- ", "").strip()
            if field_idx == 0:
                recipe["ready_in"] = text
            elif field_idx == 1:
                recipe["servings"] = text
            elif field_idx == 2:
                recipe["temperature"] = text
            field_idx += 1
            continue

        # Section headers
        if line.lower() == "ingredients":
            finalize_section()
            section = "ingredients"
            continue
        elif line.lower() == "notes":
            finalize_section()
            section = "notes"
            continue
        elif line.lower() == "preparation":  # treat this as instructions
            finalize_section()
            section = "instructions"
            continue
        elif line.lower() == "tags":
            finalize_section()
            section = "tags"
            continue

        # Collect content under current section
        if section:
            buffer.append(line)

    # Append last recipe
    if buffer:
        finalize_section()
    if recipe:
        if "tags" not in recipe:
            recipe["tags"] = []
        recipes.append(recipe)

    return recipes

if __name__ == "__main__":
    recipes = parse_recipes_from_txt("recipes.txt")
    with open("recipes.json", "w", encoding="utf-8") as f:
        json.dump(recipes, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved {len(recipes)} recipes to recipes.json")


✅ Saved 89 recipes to recipes.json
